Copyright 2018 The TensorFlow Datasets Authors, Licensed under the Apache License, Version 2.0

In [ ]:
!pip install -q tensorflow-datasets tensorflow

In [29]:
import tensorflow as tf
import tensorflow_datasets as tfds
import keras
import numpy as np
import random
#'train' is the only available split
ds = tfds.load('colorectal_histology', split='train', shuffle_files=True)


In [16]:
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Resizing, Rescaling

In [104]:
X = np.array([row['image'] for row in ds])


In [105]:
y = np.array([row['label'] for row in ds])
y = keras.utils.to_categorical(y)
z = y[0][:]

In [106]:
y = y[:,0]

# Data augmentation for tumors

In [107]:
tumor_images = X[np.where(y == 1)]
tumor_images.shape

(625, 150, 150, 3)

In [108]:
new_tumors = []
while len(new_tumors)<5000:
  random_index = random.randint(0,len(tumor_images)-1)
  random_tumor = tumor_images[random_index]
  rotated_tumor = tf.keras.preprocessing.image.random_rotation(x = random_tumor,
                                                              row_axis=0,
                                                              col_axis=1,
                                                              channel_axis=2,
                                                              rg = 360)
  new_tumors.append(rotated_tumor) 
  y = np.append(y, 1)

ary_tumors = np.array(new_tumors)

In [110]:
ary_tumors = np.array(new_tumors)
X = np.concatenate((X,ary_tumors))


Visual proof of the working rotation

In [111]:
from PIL import Image
import numpy as np

w, h = 512, 512
data = np.zeros((h, w, 3), dtype=np.uint8)
data[0:256, 0:256] = [255, 0, 0] # red patch in upper left
img = Image.fromarray(
    tf.keras.preprocessing.image.random_rotation(x = X[-1],
                                                  row_axis=0,
                                                  col_axis=1,
                                                  channel_axis=2,
                                                  rg = 360),
                       'RGB')
img.save('my.png')
img.show()

In [112]:
X.shape

(10000, 150, 150, 3)

# Model

In [113]:
model = keras.models.Sequential([
    # include preprocessing layer, along with the input shape
    tf.keras.layers.Rescaling(scale = 1/255, input_shape=(150,150,3)),
    # TODO complete the Conv2D calls
    Conv2D(16,3,padding="same",activation='relu'),
    MaxPooling2D(),
    Conv2D(32,3,padding="same",activation='relu'),
    MaxPooling2D(),
    Conv2D(64,3,padding="same",activation='relu'),
    MaxPooling2D(),
    Flatten(),
    # TODO include the final D[ense layers
    Dense(128, activation='relu'),
    Dense(1,activation='sigmoid'),
])


In [114]:
model.compile(tf.keras.optimizers.SGD(learning_rate=0.01),
              loss = tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.Recall(), "accuracy"]
)

In [115]:
(X.shape,y.shape)

((10000, 150, 150, 3), (10000,))

In [116]:
model.fit(X,y,batch_size=32, epochs=3, validation_split=.2)

Epoch 1/3
250/250 [==============================] - 5s 19ms/step - loss: 0.6885 - recall_3: 0.0830 - accuracy: 0.5446 - val_loss: 0.7648 - val_recall_3: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/3
250/250 [==============================] - 4s 17ms/step - loss: 0.6841 - recall_3: 0.0326 - accuracy: 0.5487 - val_loss: 0.8448 - val_recall_3: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 3/3
250/250 [==============================] - 4s 17ms/step - loss: 0.6806 - recall_3: 0.1230 - accuracy: 0.5625 - val_loss: 0.8095 - val_recall_3: 0.0140 - val_accuracy: 0.0140
